In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.multitasking_lib import *

In [3]:
redis_conn.hkeys('algos')

['default1', 'default', 'haikin_1', 'default3']

In [7]:
from threading import Semaphore
lock = Semaphore()

In [8]:
######################
# Initiate Backtest  #
######################
cache_type ='backtest_uni'
my_cache = cache_state('backtest_uni')
def backtest_unitest(stocks):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    algo = "haikin_1"
    fromDate = '2019-4-1'
    toDate = '2019-5-3'
    freq = '1min'
    backtest_msg={
            'stock':stocks,
            'sl':sl,
            'target':target,
            'qty':qty,
            'algo':algo,
            'fromDate':fromDate,
            'toDate':toDate,
            'freq':freq
        }

    for stock in stocks:
        redis_conn.set('stock'+cache_type,stock)
        # Step 3 : Store the OHLC and backtest data in the redis: to be used by plotter
        #ohlc_dict = '[{"date":'+json.dumps(fromDate)+',"close":0,"high":0,"low":0,"open":0,"volume":0}]'
        #redis_conn.set(stock+cache_postfix,ohlc_dict)

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)

    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))

DEBUG:	Cache initialized for: backtest_uni


In [9]:
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(logging.DEBUG)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if not backtest_manager.job.is_alive():
        backtest_manager = threadManager(cache_type, ["kite_simulator","trade_handler"], [kite_simulator, trade_handler])

    if not order_manager.job.is_alive():
        order_manager = threadManager("order", ["order_handler"], [order_handler])
except:
    backtest_manager = threadManager(cache_type, ["kite_simulator","trade_handler"], [kite_simulator, trade_handler])
    order_manager = threadManager("order", ["order_handler"], [order_handler])
    
print(backtest_manager.job.is_alive())
print(order_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


DEBUG:	Cache initialized for: backtest_uni
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: trade_handler
DEBUG:	Cache initialized for: order


True
True
['trade_handlerbacktest_web', 'kite_simulatorbacktest_web', 'order_handlerorder', 'freedom_initfreedom']
[]


DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
INFO:	trade_handler: 1
INFO:	Starting Handler: order_handler
DEBUG:	order_handler: 1
DEBUG:	kite_simulator: {"stock": ["WIPRO"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1", "fromDate": "2019-4-1", "toDate": "2019-5-3", "freq": "1min"}
DEBUG:	Trade_init: WIPRO
INFO:	WIPRObacktest_uni
TRADE: B: EN:  : 2019-04-01 09:16 : WIPRO -> 256.65
TRADE: S: EX:  : 2019-04-01 09:31 : WIPRO -> 257.5
INFO:	Kite simulator is on a short break
TRADE: B: EN:  : 2019-04-01 09:38 : WIPRO -> 257.45
TRADE: S: EX:  : 2019-04-01 09:56 : WIPRO -> 257.3
TRADE: B: EN:  : 2019-04-01 10:02 : WIPRO -> 258.2
TRADE: S: TP:  : 2019-04-01 10:12 : WIPRO -> 261.0
INFO:	Kite simulator is on a short break
TRADE: S: EN:  : 2019-04-01 10:55 : WIPRO -> 260.5
TRADE: B: EX:  : 2019-04-01 12:06 : WIPRO -> 261.4
TRADE: B: EN:  : 2019-04-01 12:15 : WIPRO -> 261.5
INFO:	Kite simulator is on a short break
TRADE: S: EX:  : 2019-04-01 12:23 : WIPRO -> 261.3
TRADE: S:

In [23]:
backtest_unitest(['WIPRO'])

In [17]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [22]:
if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()

In [20]:
backtest_manager = threadManager(cache_type, ["kite_simulator","trade_handler"], [kite_simulator, trade_handler])

DEBUG:	Cache initialized for: backtest_uni
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: trade_handler
DEBUG:	kite_simulator: 1
INFO:	trade_handler: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	kite_simulator: {"stock": ["WIPRO"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1", "fromDate": "2019-4-1", "toDate": "2019-5-3", "freq": "1min"}
DEBUG:	Trade_init: WIPRO
INFO:	WIPRObacktest_uni
TRADE: S: EN:  : 2019-04-01 09:20 : WIPRO -> 257.15
INFO:	Kite simulator is on a short break
TRADE: B: EX:  : 2019-04-01 09:58 : WIPRO -> 257.6
TRADE: S: EN:  : 2019-04-01 10:14 : WIPRO -> 260.4
TRADE: B: EX:  : 2019-04-01 10:19 : WIPRO -> 260.6
TRADE: B: EN:  : 2019-04-01 10:25 : WIPRO -> 261.45
TRADE: S: EX:  : 2019-04-01 10:55 : WIPRO -> 260.5
TRADE: B: EN:  : 2019-04-01 10:59 : WIPRO -> 260.5
INFO:	Kite simulator is on a short break
TRADE: S: EX:  : 2019-04-01 11:21 : WIPRO -> 260.3
TRADE: B: EN:  : 2019-04-01 11:26 : WIPRO -> 261.0
TRADE: S: EX:  : 2019-04-01 11:32 : WIPRO -

In [7]:
# Terminate all the processes
backtest_manager.job.terminate()
order_manager.job.terminate()
time.sleep(0.1)
print(backtest_manager.job.is_alive())
print(order_manager.job.is_alive())
print(redis_conn.pubsub_channels())
jobs
jobs = []

False
False
['trade_handlerbacktest_web', 'kite_simulatorbacktest_web', 'order_handlerorder', 'freedom_initfreedom']


In [7]:
#jobs[3].terminate()

In [ ]:
pd.DataFrame()

In [21]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
fig = freedom_chart('WIPRO')
fig

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Unit Test#1
freq='day'
exchange='NSE'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock='INFY'

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [ ]:
# Unit Test#2
stock = 'NDTV'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

In [ ]:
conn.pubsub_channels()

In [ ]:
# Debug
stock='TCS'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
my_cache = cache_state('backtest_web')

In [ ]:
df = my_cache.getValue()

In [ ]:
df

In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [29]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [30]:
ohlc_data[stock][ohlc_data[stock].index.minute+ohlc_data[stock].index.hour*60 > (15*60+15)]

,close,high,low,open,volume
date,,,,,
2019-04-22 15:16:00,288.55,288.60,288.35,288.50,52625
2019-04-22 15:17:00,288.35,288.55,288.35,288.40,24392
2019-04-22 15:18:00,288.45,288.50,288.35,288.35,44599
2019-04-22 15:19:00,288.50,288.50,288.35,288.45,38985
2019-04-22 15:20:00,288.00,289.00,287.85,288.60,199321
2019-04-22 15:21:00,288.30,288.50,287.95,287.95,45827
2019-04-22 15:22:00,288.15,288.45,288.00,288.10,57890
2019-04-22 15:23:00,288.25,288.40,288.05,288.15,42309
2019-04-22 15:24:00,288.05,288.30,288.05,288.30,15712


In [35]:
ohlc_data[stock]['close']

date
2018-12-03    246.80
2018-12-04    251.84
2018-12-05    252.97
2018-12-06    249.36
2018-12-07    250.60
               ...  
2019-02-22    284.77
2019-02-25    289.77
2019-02-26    291.47
2019-02-27    281.50
2019-02-28    277.37
Name: close, Length: 63, dtype: float64

In [3]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')

DEBUG:	Cache initialized for: backtest_web


In [4]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,0.5,1,21479.0,0,19.49999999999818,0.9070400260482444,-69.49999999999818,-0.3235718608873699,...,2163.0,0.0,0,2165.5,2019-04-23 15:29,SCANNING,PAUSE,haikin_minmax,minute,day
1,WIPRO,10,0.5,1,2898.0,0,-0.5000000000001137,-0.17250301880286828,-18.500000000000796,-0.638371290545231,...,291.6,0.0,0,291.85,2019-04-23 15:29,SCANNING,PAUSE,haikin_minmax,minute,day


In [3]:
live_cache.remove()

In [4]:
df = live_cache.getOHLC('WIPRO')

In [5]:
df.head(10)

,close,high,low,open,volume
2018-11-30,244.10,245.41,235.86,236.88,6371721
2018-12-03,246.80,247.74,242.89,244.36,2668224
2018-12-04,251.84,256.58,244.89,246.80,5971913
2018-12-05,252.97,254.66,246.73,250.38,3849080
2018-12-06,249.36,252.97,247.37,250.56,2337100
2018-12-07,250.60,252.89,243.95,250.23,3474102
2018-12-10,245.56,253.01,244.44,246.84,3198111
2018-12-11,246.62,250.98,244.21,244.66,4423215
2018-12-12,248.16,249.62,246.58,247.97,2635669
2018-12-13,254.62,258.61,247.37,248.87,5148093


In [28]:
# Resampling
freq = '2T'

tmp_df = pd.DataFrame()

tmp_df['close'] = df['close'].resample(freq,label='right', closed='right').last()
tmp_df['high'] = df['high'].resample(freq,label='right', closed='right').max()
tmp_df['low'] = df['low'].resample(freq,label='right', closed='right').min()
tmp_df['open'] = df['open'].resample(freq,label='right', closed='right').first()

In [140]:
msg_ltp = {
    "status": "success",
    "data": {
        "BSE:SENSEX": {
            "instrument_token": 265,
            "last_price": 31606.48
        },
        "NSE:INFY": {
            "instrument_token": 408065,
            "last_price": 890.9
        },
        "NSE:NIFTY 50": {
            "instrument_token": 256265,
            "last_price": 9893.4
        }
    }
}

In [139]:
#json.dumps(msg_ltp)

In [145]:
count = 0
live_cache.xadd('tickBuffer',{"status":"success", "data":})

DataError: Invalid input of type: 'dict'. Convert to a bytes, string, int or float first.

In [77]:
live_cache.xlen('tickBuffer')

6

In [86]:
live_cache.xrange('tickBuffer','1588796114836-0')

[('1588796114836-0',
  {'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}),
 ('1588796116726-0',
  {'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}),
 ('1588796118122-0',
  {'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}),
 ('1588796119457-0',
  {'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {

In [58]:
live_cache.xread({'tickBuffer':'$'},count=100)

[]

In [87]:
live_cache.xread({'tickBuffer':'$'},count=100)

[]

In [91]:
tickBuffer = live_cache.xread({'tickBuffer':'0-0'})

In [147]:
live_cache.xtrim('tickBuffer',0)

8

In [ ]:
'1588796103781-0'

In [124]:
for i in payload:
    print(payload[i])

{'instrument_token': 265, 'last_price': 31606.48}
{'instrument_token': 408065, 'last_price': 890.9}
{'instrument_token': 256265, 'last_price': 9893.4}


In [122]:
payload = json.loads(tickBuffer[0][1][0][1]['msg'])['data']

In [146]:
for i,val in enumerate(tickBuffer[0][1]):
    print(val[1])

{'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}
{'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}
{'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}
{'msg': '{"status": "success", "data": {"BSE:SENSEX": {"instrument_token": 265, "last_price": 31606.48}, "NSE:INFY": {"instrument_token": 408065, "last_price": 890.9}, "NSE:NIFTY 50": {"instrument_token": 256265, "last_price": 9893.4}}}'}
{'msg': '{"status": "success", "data": {"BSE